In [25]:
%load_ext autoreload
%autoreload 2
from funlib.persistence import open_ds
import matplotlib.pyplot as plt
import numpy as np
import gunpowder as gp
import torch
from src.datasets import load_dataset, Kh2015, TorchZarrDataloader
from src.transforms import PreprocessingPipeline
from src.config import read_config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
dataset = Kh2015(
  transform=read_config("examples/no_augments")["pipeline"]
)

input_shape = (36, 212, 212)
output_shape = (12, 120, 120)

dataloader = TorchZarrDataloader(
  dataset=dataset,
  input_shape=input_shape,
  output_shape=output_shape,
)

def collate(x):
  num_return_types = len(x[0])
  return *[
    np.stack([y[i] for y in x])
    for i in range(num_return_types)
  ],

dataloader = torch.utils.data.DataLoader(
  dataloader,
  batch_size=4,
  collate_fn=collate,
  num_workers=5,
  prefetch_factor=4,
)

it = iter(dataloader)
raw, _ = next(it)
raw.shape

Dataset already found on disk, downloading...
(50, 2, 2)(50, 2, 2)

<src.transforms.gp_transform.PreprocessingPipeline object at 0x7f49b42f4c40><src.transforms.gp_transform.PreprocessingPipeline object at 0x7f49b42f4c40>

(50, 2, 2)
(50, 2, 2)
(50, 2, 2)


(4, 36, 212, 212)

In [23]:
sample, labels, affs, affs_weights = next(ds_it)
sample, labels, affs, affs_weights = sample[0][0], labels[0], affs[0], affs_weights[0]
start_end =  (212-120)//2
sample = sample[18,start_end:-start_end,start_end:-start_end]
labels = labels[6]
affs = affs[:,6,:,:].transpose(1, 2, 0).astype(np.float32)
print(labels.shape)
print(affs.shape)
print(affs[0,0])
#print(affs)
# show 2 images both sample and labels
fig, ax = plt.subplots(3, 1, figsize=(15, 10))
ax[0].axis("off")
ax[0].imshow(sample, cmap="gray")
ax[1].axis("off")
ax[1].imshow(labels, cmap="jet")
ax[2].axis("off")
ax[2].imshow(affs)
plt.show()

RuntimeError: Don't know how to add <class 'NoneType'> to Pipeline

In [ ]:
#roi = gp.Coordinate(ds.shape) / 2 * ds.voxel_size 
#sample, *_ = ds[roi]
#plt.axis("off")
#plt.imshow(sample, cmap="gray");